# Cel zadania i problem biznesowy


# Preprocessing

Nasz zbiór danych w postaci .csv zawierał aż 33 kolumny, co wiązało się z faktem że nasz preprocessing był dość obszerny. Do każdej kolumny staraliśmy się podejść indywidualnie i odpowiednio przetransformować jej wartości, aby póżniej algorytm uczenia maszynowego działał poprawnie. 

W przypadku kolumn kategorycznych zazwyczaj decydowaliśmy się na użycie algorytmu OneHotEncoding z pakietu scikit-learn. Jest to dość intuicyjna metoda, dzięki której nie tracimy żadnych informacji. Dodaje on do zbioru danych dodatkowe kolumny (ich liczność jest odpowiednia liczbą kategorii), a następnie daje wartość 1 tam, gdzie jest odpowiednia kategoria. Sprawia to, że informacje te są zakodowane w postaci liczbowej. Niestety sprawia to, że dodajemy wiele dodatkowych kolumn, co negatywnie wpływa na interpretowalność modelu zbudowanego na tej podstawie. Przykładowymi kolumnami kategorycznymi, na których zastosowaliśmy ten algorytm to: 'TopThreeAmericanName', 'WheelTypeID','Nationality', itd... 

Kolejnym ważnym etapem preprocessingu było zastosowanie normalizacji danych. Zważając na fakt, że budowaliśmy modele które są czułe na skale zmiennych (np. SVM) musieliśmy wszystkie wartości liczbowe w jakiś sposób przeskalować, aby były między sobą 'porównywalne'. Zastosowaliśmy funkcję StandardScaler, która odejmuje od każdej wartości średnią arytmetyczną (mean), a następnie dzieli przez wariancję. Przykładowymi kolumnami objętymi tym algorytmem są: 'VehOdo','VehBCost', 'WarrantyCost', itd ... 



Tutaj jeszcze bardzo warto wspomnieć o tym wheelID ze sie daje to dużo


# Modele

## Dummy classifier


Aby sprawdzić, czy nasze dane są dobrze przygotowane do trenowania modelów wytrenowaliśmy *dummy classifier*, który przyporządkowuje po prostu jedną z dwóch kategorii do obserwacji.

## KNN classifier

moze tutaj opisac troche jak działa ten model ale nie wiem czy to konieczne

Po wykonaniu hiperparametryzacji za pomocą metody grid search sprawdziliśmy jak sprawuje się model na zbiorze walidacyjnym

![roc curve](plots/KNN_roc.png)

Krzywa ROC - jak widać model nie sprawuje się zbyt dobrze. Otrzymana wartość gini dla zbioru walidacyjnego to zaledwie 0.3332

Model KNN sprawuje się gorzej przy bardzo wielu wymiarach. W związku z tym postanowiliśmy zredukować liczbę wymiarów. Z pomocą przyszedł nam algorytm PCA, który zastosowaliśmy.


Aby zachować dostatecznie dużo informacji postanowliśmy wybrać liczbę wymiarów PCA, tak aby łączna wariancja miała wartość co najmniej 95%.

![pca](plots/PCA_variance.png)


Okazało się jednak, że zastosowana redukcja wymiarów nie przyniosła satysfakcjonujących rezultatów, gdyż model po redukcji wymiarów sprawował się gorzej.

![roc curve](plots/KNN_pca_roc.png)



Confusion matrix przed redukcją wymiarów:

![conf_knn](plots/KNN_confusion.png)


Wynik algorytmu na danych testowych z kaggla:

![knn_res](plots/KNN_res.png)


**Ogólnie model ten poradził sobie raczej słabo**.


## Random forest

Jednym z najczęściej używanych modeli jest las lasowy. Również postanowiliśmy zaimplementować ten model do rozwiązania tego problemu.


Przy użyciu standardowych hiperparametrów wytrenenowaliśmy pierwszy model tego typu, sprawował się on znacznie lepiej niż poprzednie:


![roc curve](plots/RF_base_roc.png)

Wartość gini dla tego modelu wynosiła około 0.5



Przy użyciu podstawowych metod wyszukiwania najlepszych hiperparametrów, takich jak grid oraz random search wytrenowaliśmy model sprawujący się lepiej na danych treningowych


![roc curve](plots/RF_random_roc.png)

Wartość gini dla tego modelu wynosiła również około 0.5, ale mniej niż w poprzednim przypadku.



W ramach tego modelu sprawdziliśmy również feature importance na dwa sposoby - za pomocą wbudowanej funkcji lasu losowego `feature_importance_`, a także za pomocą pakietu `shapely`.

Jak się okazało najbardziej istotną kolumną predykcyjną była kolumna oznaczająca brak danych w kolumnie WheelTypeID. W zależności od parametrów, predykcyjność tej kolumny wynosiła nawet do 60%.






## XGBoost

Zdecydowaliśmy się ponownie użyć grid-search'a, aby znaleźć najlepsze hiperparametry modelu XGboost. \
Jak się okazało najlepszy model przy cross-validation=3 sprawdził się XGBoost z następującymi parametrami: 

{'subsample': 1.0,
 'n_estimators': 150,
 'max_depth': 5,
 'learning_rate': 0.1,
 'gamma': 5,
 'colsample_bytree': 1.0}
 
 Dla modelu z powyższymi parametrami obliczyliśmy różne metryki, które przedstawiają się następująco. \
 Dla klasy 0: precision: 0.88, recall: 0.99 \
 Dla klasy 1: precision: 0.39, recall: 0.02 \
 Accuracy: 0.88

A jeżeli chodzi o macierz pomyłek to wygląda ona następująco:

![Confusion Matrix](plots/xgb_confusion.png)

A krzywe roc tak:

Dla danych treningowych:

![Roc xgb](plots/roc-xgb-train.png)

Gini wynosi tutaj 0.67

Dla danych testowych:

![Roc xgb test](plots/roc-xgb-test.png)

A tutaj wartość gini 0.3

Co rzuciło nam się w oczy to fakt, że na danych treningowych wartość gini jest większa niż w przypadku Random Forest. Z drugiej strony, dla danych testowych sytuacja ma się odwrotnie. 
